In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression

In [9]:
data = pd.read_csv("/kaggle/input/breast-cancer-prediction-dataset/Breast_cancer_data.csv") 

In [10]:
data.head()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0
1,20.57,17.77,132.90,1326.0,0.08474,0
2,19.69,21.25,130.00,1203.0,0.10960,0
3,11.42,20.38,77.58,386.1,0.14250,0
4,20.29,14.34,135.10,1297.0,0.10030,0


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   mean_radius      569 non-null    float64
 1   mean_texture     569 non-null    float64
 2   mean_perimeter   569 non-null    float64
 3   mean_area        569 non-null    float64
 4   mean_smoothness  569 non-null    float64
 5   diagnosis        569 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 26.8 KB


In [13]:
data.diagnosis = [1 if each == '0' else 0 for each in data.diagnosis]

In [14]:
y = data.diagnosis.values #values methodu ile numpy array'e çevrilir.
x_data = data.drop(["diagnosis"],axis=1)

In [16]:
x = (x_data-np.min(x_data))/(np.max(x_data)-np.min(x_data)).values

In [17]:
x

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness
0,0.521037,0.022658,0.545989,0.363733,0.593753
1,0.643144,0.272574,0.615783,0.501591,0.289880
2,0.601496,0.390260,0.595743,0.449417,0.514309
3,0.210090,0.360839,0.233501,0.102906,0.811321
4,0.629893,0.156578,0.630986,0.489290,0.430351
...,...,...,...,...,...
564,0.690000,0.428813,0.678668,0.566490,0.526948
565,0.622320,0.626987,0.604036,0.474019,0.407782
566,0.455251,0.621238,0.445788,0.303118,0.288165
567,0.644564,0.663510,0.665538,0.475716,0.588336


In [18]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split (x,y,test_size=0.20,random_state=42)

In [11]:
x_train = x_train.T
x_train.shape

(30, 455)

In [19]:
x_test = x_test.T
y_train = y_train.T
y_test = y_test.T

In [13]:
print(x_test.shape)

(30, 114)


In [20]:
y_train.shape

(455,)

In [21]:
y_test.shape

(114,)

In [27]:
def initialize_weights_and_bias(dimension):
    w = np.full((dimension,1),0.01)
    b=0.0
    return w,b

In [28]:
def sigmoid(z):
    y_head = 1/(1+np.exp(-z))
    return y_head

In [29]:
sigmoid(0)

0.5

In [31]:
def forward_backward_propagation(w,b,x_train,y_train):

    # forward propagation
    z = np.dot(w.T,x_train) + b 
    y_head = sigmoid(z) 
    loss = -y_train*np.log(y_head)-(1-y_train)*np.log(1-y_head) 
    cost = (np.sum(loss))/x_train.shape[1]   

    # backward propagation
    derivative_weight = (np.dot(x_train,((y_head-y_train).T)))/x_train.shape[1]  
    derivative_bias = np.sum(y_head-y_train)/x_train.shape[1]              
    gradients = {"derivative_weight": derivative_weight,"derivative_bias": derivative_bias}

    return cost,gradients

In [32]:
def update(w, b, x_train, y_train, learning_rate,number_of_iterarion):
    cost_list = []
    cost_list2 = []
    index = []
    
    for i in range(number_of_iterarion): 
        cost,gradients = forward_backward_propagation(w,b,x_train,y_train) 
        cost_list.append(cost) 
        w = w - learning_rate * gradients["derivative_weight"]  
        b = b - learning_rate * gradients["derivative_bias"]   
        if i % 10 == 0:
            cost_list2.append(cost) 
            index.append(i)
            print ("Cost after iteration %i: %f" %(i, cost))

    parameters = {"weight": w,"bias": b} 
    plt.plot(index,cost_list2) 
    plt.xticks(index,rotation='vertical') 
    plt.xlabel("Number of Iterarion")
    plt.ylabel("Cost")
    plt.show()
    return parameters, gradients, cost_list 

In [33]:
def predict(w,b,x_test):
    
    z = sigmoid(np.dot(w.T,x_test)+b) 
    Y_prediction = np.zeros((1,x_test.shape[1])) #(1,114) 
    
    for i in range(z.shape[1]): 
        if z[0,i]<= 0.5: 
            Y_prediction[0,i] = 0
        else:
            Y_prediction[0,i] = 1
                        
    return Y_prediction

In [37]:

def logistic_regression(x_train, y_train, x_test, y_test, learning_rate, num_iterations):
    # Ensure x_train and y_train have the same number of samples
    assert x_train.shape[0] == y_train.shape[0], "x_train and y_train must have the same number of samples"
    # Ensure x_test and y_test have the same number of features
    assert x_test.shape[1] == x_train.shape[1], "x_test and x_train must have the same number of features"
    # Ensure y_train and y_test are 1-dimensional arrays
    assert y_train.ndim == 1 and y_test.ndim == 1, "y_train and y_test must be 1-dimensional arrays"

    dimension = x_train.shape[0]
    w, b = initialize_weights_and_bias(dimension)

    parameters, gradients, cost_list = update(w, b, x_train, y_train, learning_rate, num_iterations)

    y_prediction_test = predict(parameters["weight"], parameters["bias"], x_test)

    print("test accuracy: {} %".format(100 - np.mean(np.abs(y_prediction_test - y_test)) * 100))



In [ ]:
from sklearn.linear_model import LogisticRegression

# Create an instance of LogisticRegression
lr = LogisticRegression(random_state=42, max_iter=40)

# Fit the model to the training data
lr.fit(x_train, y_train)

In [ ]:
y_pred = lr.predict(x_test.T)

In [ ]:
print("test accuracy {}".format(lr.score(x_test.T,y_test.T)))